In [1]:
import dxpy 
import dxdata 
import pandas as pd
import pyspark
import re

In [2]:
# Initialize Spark
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed dataset ID and load the dataset
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", 
    name="app*.dataset", 
    folder="/", 
    name_mode="glob")
dispensed_dataset_id = dispensed_dataset["id"]
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [4]:
participant = dataset['participant']

In [9]:
# load cohorts from cohort browser
case = dxdata.load_cohort("/HTN_ICD10/ICD10_HTN_Cases")  
cont = dxdata.load_cohort("/HTN_ICD10/ICD10_HTN_Control")  

In [10]:
# Specify fields ID to retrieve, get corresponding UKB RAP field names and print description table.
field_ids = ['31', '22001', '22006', '22019', '22021', '21022', '41270']

In [11]:
# This function is used to grab all field names (e.g. "p<field_id>_iYYY_aZZZ") of a list of field IDs
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

In [12]:
fields = [fields_for_id(f)[0] for f in field_ids] + [participant.find_field(name='p20160_i0')] + [participant.find_field(name='eid')]
field_description = pd.DataFrame({
    'Field': [f.name for f in fields],
    'Title': [f.title for f in fields],
    'Coding': [f.coding.codes if f.coding is not None else '' for f in fields ]
 })
field_description

/tmp/ipykernel_90/40657072.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return sorted(fields, key=lambda f: LooseVersion(f.name))


,Field,Title,Coding
0,p31,Sex,"{'0': 'Female', '1': 'Male'}"
1,p22001,Genetic sex,"{'0': 'Female', '1': 'Male'}"
2,p22006,Genetic ethnic grouping,{'1': 'Caucasian'}
3,p22019,Sex chromosome aneuploidy,{'1': 'Yes'}
4,p22021,Genetic kinship to other participants,{'-1': 'Participant excluded from kinship infe...
5,p21022,Age at recruitment,
6,p41270,Diagnoses - ICD10,{'Chapter I': 'Chapter I Certain infectious an...
7,p20160_i0,Ever smoked | Instance 0,"{'1': 'Yes', '0': 'No'}"
8,eid,Participant ID,


In [13]:
case_df = participant.retrieve_fields(fields = fields, filter_sql = case.sql, engine=dxdata.connect()).toPandas()
cont_df = participant.retrieve_fields(
    fields = fields, filter_sql = cont.sql,
    engine=dxdata.connect(
    dialect="hive+pyspark", 
        connect_args=
        {
            'config':{'spark.kryoserializer.buffer.max':'256m','spark.sql.autoBroadcastJoinThreshold':'-1'}
                     
        }
)).toPandas()

In [14]:
df = pd.concat([case_df, cont_df])

In [15]:
df.shape

(502137, 9)

In [20]:
df['HTN_cc'] = 0
df.loc[df.eid.isin(case_df.eid),'HTN_cc'] = 1

In [21]:
df.HTN_cc.value_counts()

0    339876
1    162261
Name: HTN_cc, dtype: int64

In [22]:
# Sample QC
# Gender and genetic sex are the same, white british ancestry, no sex chromosome aneuploidy, no kinship found
df_qced = df[
    (df['p31'] == df['p22001']) & # Filter in sex and genetic sex are the same           
    (df['p22006'] == 1) &         # in_white_british_ancestry_subset           
    (df['p22019'].isnull()) &     # Not Sex chromosome aneuploidy           
    (df['p22021'] == 0)           # No kinship found
]

In [23]:
df_qced.HTN_cc.value_counts()

0    188930
1     87132
Name: HTN_cc, dtype: int64

In [24]:
# Rename columns for better readibility and format table for regenie
df_qced = df_qced.rename(columns=
                         {'eid':'IID', 'p31': 'sex', 'p21022': 'age',
                          'p20160_i0': 'ever_smoked',
                          'p22006': 'ethnic_group',                           
                          'p22019': 'sex_chromosome_aneuploidy',                          
                          'p22021': 'kinship_to_other_participants'})
# Add FID column -- required input format for regenie 
df_qced['FID'] = df_qced['IID']

# Create a phenotype table from our QCed data
df_phenotype = df_qced[['FID', 'IID', 'HTN_cc', 'sex', 'age', 'ethnic_group', 'ever_smoked']]

In [27]:
# Intersect with WES dataset to generate phenotype file 
# Get WES 
# Merge with WES data to leave only participants with WES data available
# Get WES
path_to_family_file = f'/mnt/project/Bulk/Exome sequences_Previous exome releases/Population level exome OQFE variants, PLINK format - interim 200k release/ukb23155_cY_b0_v1.fam'
plink_fam_df = pd.read_csv(path_to_family_file, delimiter='\s', dtype='object',                           
                           names = ['FID','IID','Father ID','Mother ID', 'sex', 'Pheno'], engine='python')
# Intersect the phenotype file and the 200K WES .fam file
# to generate phenotype DataFrame for the 200K participants
# filtering out only those that have whole exome available 
HTN_wes_200k_df = df_phenotype.join(plink_fam_df.set_index('IID'), on='IID', rsuffix='_fam', how='inner')
# Drop unuseful columns from .fam file
HTN_wes_200k_df.drop(
    columns=['FID_fam','Father ID','Mother ID','sex_fam', 'Pheno'], axis=1, inplace=True, errors='ignore'
)

In [28]:
# Write phenotype files to a TSV file
HTN_wes_200k_df.to_csv('HTN_wes_200k.phe', sep='\t', na_rep='NA', index=False, quoting=3)
df_phenotype.to_csv('HTN_wes.phe', sep='\t', na_rep='NA', index=False, quoting=3)